In [17]:
import numpy as np
import pandas as pd
import requests
import sqlite3
from pathlib import Path


In [18]:
polling = pd.read_csv("../president_primary_polls.csv")


/home/octav/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
np.sort

<function numpy.sort(a, axis=-1, kind=None, order=None)>

In [4]:
# polling["state"].unique() list(map(lambda x: "National" if x != x else x, polling["state"].unique()))

In [5]:
polling["state"][0] == polling["state"][0]

False

In [15]:
url = 'https://projects.fivethirtyeight.com/polls-page/president_primary_polls.csv'
r = requests.get(url, allow_redirects=True)

In [16]:
with open("../president_primary_polls.csv", 'wb') as f:
    f.write(r.content)

In [21]:
polling_subset = polling[polling["party"] == "DEM"]
polling["create_date"] = pd.to_datetime(polling["created_at"], infer_datetime_format=True)
polling["create_date"]
polling["create_week"] = (polling['create_date'] - pd.to_timedelta(polling['create_date'].dt.dayofweek, unit='d')
                          + np.timedelta64(7, 'D')).dt.normalize()                                                             
polling_subset = polling[polling["party"] == "DEM"]
df_pivot = polling[polling["party"] == "DEM"].pivot_table(values="pct", index="create_week", columns="answer",
                                                          aggfunc=np.mean).fillna(0).iloc[-7:]
candidates = polling_subset["answer"].unique()


In [22]:
polling

,question_id,poll_id,cycle,state,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,...,notes,url,stage,party,answer,candidate_id,candidate_name,pct,create_date,create_week
0,118856,64512,2020,South Carolina,383,Public Policy Polling,NaN,NaN,Public Policy Polling,263.0,...,NaN,https://www.publicpolicypolling.com/wp-content...,primary,DEM,Biden,13256,Joseph R. Biden Jr.,36.0,2020-02-24 15:14:00,2020-03-02
1,118856,64512,2020,South Carolina,383,Public Policy Polling,NaN,NaN,Public Policy Polling,263.0,...,NaN,https://www.publicpolicypolling.com/wp-content...,primary,DEM,Buttigieg,13345,Pete Buttigieg,7.0,2020-02-24 15:14:00,2020-03-02
2,118856,64512,2020,South Carolina,383,Public Policy Polling,NaN,NaN,Public Policy Polling,263.0,...,NaN,https://www.publicpolicypolling.com/wp-content...,primary,DEM,Gabbard,13343,Tulsi Gabbard,6.0,2020-02-24 15:14:00,2020-03-02
3,118856,64512,2020,South Carolina,383,Public Policy Polling,NaN,NaN,Public Policy Polling,263.0,...,NaN,https://www.publicpolicypolling.com/wp-content...,primary,DEM,Klobuchar,13310,Amy Klobuchar,3.0,2020-02-24 15:14:00,2020-03-02
4,118856,64512,2020,South Carolina,383,Public Policy Polling,NaN,NaN,Public Policy Polling,263.0,...,NaN,https://www.publicpolicypolling.com/wp-content...,primary,DEM,Sanders,13257,Bernard Sanders,21.0,2020-02-24 15:14:00,2020-03-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17326,92392,57184,2020,NaN,1416,Harris Insights & Analytics,960,Hill.TV,HarrisX,133.0,...,NaN,https://thehill.com/hilltv/what-americas-think...,primary,DEM,Sanders,13257,Bernard Sanders,20.0,2018-12-27 03:50:00,2018-12-31
17327,92392,57184,2020,NaN,1416,Harris Insights & Analytics,960,Hill.TV,HarrisX,133.0,...,NaN,https://thehill.com/hilltv/what-americas-think...,primary,DEM,Booker,13287,Cory A. Booker,5.0,2018-12-27 03:50:00,2018-12-31
17328,92392,57184,2020,NaN,1416,Harris Insights & Analytics,960,Hill.TV,HarrisX,133.0,...,NaN,https://thehill.com/hilltv/what-americas-think...,primary,DEM,Harris,13288,Kamala D. Harris,5.0,2018-12-27 03:50:00,2018-12-31
17329,92392,57184,2020,NaN,1416,Harris Insights & Analytics,960,Hill.TV,HarrisX,133.0,...,NaN,https://thehill.com/hilltv/what-americas-think...,primary,DEM,Bloomberg,13289,Michael Bloomberg,5.0,2018-12-27 03:50:00,2018-12-31


In [23]:
polling_subset[["answer", "candidate_id"]]

,answer,candidate_id
0,Biden,13256
1,Buttigieg,13345
2,Gabbard,13343
3,Klobuchar,13310
4,Sanders,13257
...,...,...
17326,Sanders,13257
17327,Booker,13287
17328,Harris,13288
17329,Bloomberg,13289


In [24]:
can_selected = []
for can in candidates:
    if (df_pivot[can] == 0).mean() < 0.5 and df_pivot[can].mean() > 0.1:
        print(can)
        can_selected.append(can)


Biden
Buttigieg
Gabbard
Klobuchar
Sanders
Steyer
Warren
Bloomberg
Bennet
Patrick
Yang
Booker
Delaney


In [25]:
print('Beginning file download with requests')

for can in can_selected:
    url = 'https://api.gdeltproject.org/api/v2/tv/tv?query="' + can + '"%20market:%22National%22&mode=timelinevol&format' \
                                                                      '=csv&datanorm=perc&dateres=week&timelinesmooth=0' \
                                                                      '&datacomb=sep' \
                                                                      '&last24=yes&timezoom=yes '
    r = requests.get(url)

    with open('candidates_coverage/' + can + '.csv', 'wb') as f:
        f.write(r.content)

    print(url)
    print(r.status_code)
    print(r.headers['content-type'])
    print(r.encoding)


Beginning file download with requests
https://api.gdeltproject.org/api/v2/tv/tv?query="Biden"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8
https://api.gdeltproject.org/api/v2/tv/tv?query="Buttigieg"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8
https://api.gdeltproject.org/api/v2/tv/tv?query="Gabbard"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8
https://api.gdeltproject.org/api/v2/tv/tv?query="Klobuchar"%20market:%22National%22&mode=timelinevol&format=csv&datanorm=perc&dateres=week&timelinesmooth=0&datacomb=sep&last24=yes&timezoom=yes 
200
text/csv; charset=utf-8
utf-8
https://api.gdeltproject.org/api/v2/tv/tv?query="Sanders

In [26]:
source_files = sorted(Path('candidates_coverage').glob('*.csv'))

dataframes = []
for file in source_files:
    df = pd.read_csv(file)
    df['Candidate'] = file.name.replace(".csv", "")
    dataframes.append(df)

media_coverage = pd.concat(dataframes, sort=False)
media_coverage.rename(columns={(media_coverage.columns[0]): 'Date'}, inplace=True)
media_coverage['Date'] = pd.to_datetime(media_coverage['Date'])
media_coverage.to_csv('coverage.csv')
# media_coverage.to_csv('C:\\Users\\User\\PycharmProjects\\WebDB\\polls\\media_coverage.csv')


In [29]:
media_coverage["Date"]

1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
1   2009-06-07
Name: Date, dtype: datetime64[ns]

In [ ]:
polling_means = polling_subset.groupby(['answer', 'create_week'])['pct'].mean().reset_index()
conn = sqlite3.connect(':memory:')
media_coverage.to_sql('media_coverage', conn, index=False)
polling_means.to_sql('polling_means', conn, index=False)
qry = '''
    SELECT *
    FROM
        polling_means JOIN media_coverage ON
        Candidate = answer
        AND create_week BETWEEN Date AND date(Date, '+7 day')
    '''
corr_data = pd.read_sql_query(qry, conn)
# corr_data.to_csv('../polls/corr_data.csv')
